## 准备数据

In [12]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [13]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [14]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([784, 256], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.normal([256, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [15]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [16]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.9686787 ; accuracy 0.13141666
epoch 1 : loss 2.8371105 ; accuracy 0.13651666
epoch 2 : loss 2.7359526 ; accuracy 0.14081667
epoch 3 : loss 2.655664 ; accuracy 0.1464
epoch 4 : loss 2.5902052 ; accuracy 0.15163334
epoch 5 : loss 2.535541 ; accuracy 0.1568
epoch 6 : loss 2.4888902 ; accuracy 0.16328333
epoch 7 : loss 2.4482822 ; accuracy 0.1694
epoch 8 : loss 2.4123077 ; accuracy 0.1765
epoch 9 : loss 2.3799403 ; accuracy 0.18436667
epoch 10 : loss 2.350429 ; accuracy 0.19286667
epoch 11 : loss 2.323206 ; accuracy 0.20011666
epoch 12 : loss 2.297841 ; accuracy 0.20845
epoch 13 : loss 2.2740085 ; accuracy 0.21715
epoch 14 : loss 2.2514567 ; accuracy 0.22701667
epoch 15 : loss 2.2299855 ; accuracy 0.2355
epoch 16 : loss 2.2094364 ; accuracy 0.24365
epoch 17 : loss 2.1896844 ; accuracy 0.25181666
epoch 18 : loss 2.1706264 ; accuracy 0.26015
epoch 19 : loss 2.1521795 ; accuracy 0.26911667
epoch 20 : loss 2.1342754 ; accuracy 0.27835
epoch 21 : loss 2.1168563 ; accuracy 0.286